In [1]:
import geopandas as gpd

In [21]:
data = gpd.read_file(r"C:\Users\meloy\SW2020 Dropbox\SW2020\Workspaces\Narek_Meloyan\Tarana Project\Data-Structured\Alabama\Alabama BEAD Grid Analysis Layer.sqlite")

In [22]:
data.head()

,index,point_count,geometry
0,0,0,"POLYGON ((-88.46548 30.22369, -88.46548 30.233..."
1,1,0,"POLYGON ((-88.45548 30.22369, -88.45548 30.233..."
2,2,0,"POLYGON ((-88.44548 30.22369, -88.44548 30.233..."
3,3,0,"POLYGON ((-88.43548 30.22369, -88.43548 30.233..."
4,4,0,"POLYGON ((-88.42548 30.22369, -88.42548 30.233..."


In [23]:
# Define the range dictionary
range_dict = {
    "0": {"range": (1, 5), "color": "#e4e4f3"},
    "1": {"range": (5, 10), "color": "#d1d1ea"},
    "2": {"range": (10, 20), "color": "#b3b3e0"},
    "3": {"range": (20, 30), "color": "#8080c5"},
    "4": {"range": (30, 50), "color": "#6d6dbd"},
    "5": {"range": (50, 75), "color": "#4949ac"},
    "6": {"range": (75, 100), "color": "#3737a4"},
    "7": {"range": (100, 50000), "color": "#121293"},
}

In [24]:
# Create detailed statistics and save to Excel (updated version)
import pandas as pd


# Function to classify point_count into range classes
def classify_point_count(point_count):
    for class_key, class_info in range_dict.items():
        min_val, max_val = class_info["range"]
        if min_val <= point_count < max_val:
            return class_key
    return "unclassified"


# Add classification column
data['range_class'] = data['point_count'].apply(classify_point_count)

# Create detailed statistics
summary_data = []
total_entries = len(data)
total_point_count = data['point_count'].sum()

for class_key, class_info in range_dict.items():
    min_val, max_val = class_info["range"]
    class_data = data[data['range_class'] == class_key]
    count = len(class_data)
    percentage = (count / total_entries) * 100 if total_entries > 0 else 0

    # Sum of point_count for this class
    point_count_sum = class_data['point_count'].sum()
    point_count_percentage = (point_count_sum / total_point_count) * 100 if total_point_count > 0 else 0

    # Display range properly (subtract 1 from max for inclusive range display)
    display_max = max_val - 1 if max_val != 50000 else "50000+"
    range_display = f"{min_val} - {display_max}"

    summary_data.append({
        'Class': int(class_key),
        'Range': range_display,
        'Entry_Count': count,
        'Entry_Percentage': round(percentage, 2),
        'Point_Count_Sum': point_count_sum,
        'Point_Count_Percentage': round(point_count_percentage, 2)
    })

# Handle unclassified entries
unclassified_data = data[data['range_class'] == 'unclassified']
if len(unclassified_data) > 0:
    count = len(unclassified_data)
    percentage = (count / total_entries) * 100
    point_count_sum = unclassified_data['point_count'].sum()
    point_count_percentage = (point_count_sum / total_point_count) * 100 if total_point_count > 0 else 0

    summary_data.append({
        'Class': 'Unclassified',
        'Range': 'Outside defined ranges',
        'Entry_Count': count,
        'Entry_Percentage': round(percentage, 2),
        'Point_Count_Sum': point_count_sum,
        'Point_Count_Percentage': round(point_count_percentage, 2)
    })

# Create DataFrame
summary_df = pd.DataFrame(summary_data)

print("Summary Statistics:")
print(summary_df)
print(f"\nTotal entries: {total_entries}")
print(f"Total point count: {total_point_count}")

Summary Statistics:
          Class                   Range  Entry_Count  Entry_Percentage  \
0             0                   1 - 4        21035             12.44   
1             1                   5 - 9         5525              3.27   
2             2                 10 - 19         3024              1.79   
3             3                 20 - 29          781              0.46   
4             4                 30 - 49          444              0.26   
5             5                 50 - 74          156              0.09   
6             6                 75 - 99           63              0.04   
7             7            100 - 50000+           69              0.04   
8  Unclassified  Outside defined ranges       137990             81.61   

   Point_Count_Sum  Point_Count_Percentage  
0            39132                   22.22  
1            36275                   20.60  
2            40120                   22.78  
3            18468                   10.49  
4            1

In [25]:
# Save to Excel with proper formatting (updated version)
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter

# Create workbook and worksheet
wb = Workbook()
ws = wb.active
ws.title = "Point Count Analysis"

# Add title
ws['A1'] = "Point Count Distribution Analysis"
ws['A1'].font = Font(bold=True, size=16)
ws['A1'].alignment = Alignment(horizontal='center')
ws.merge_cells('A1:F1')

# Add summary info
ws['A3'] = f"Total Entries (1 Km Sq. Grid]: {total_entries}"
ws['A3'].font = Font(bold=True)
ws['A4'] = f"Total BSL Count: {total_point_count}"
ws['A4'].font = Font(bold=True)
ws['A5'] = f"Analysis Date: {pd.Timestamp.now().strftime('%Y-%m-%d')}"

# Add headers starting from row 7
headers = ['Class', 'Range', 'Entry Count', 'Entry Percentage (%)', 'Point Count Sum', 'Point Count Percentage (%)']
for col, header in enumerate(headers, 1):
    cell = ws.cell(row=7, column=col, value=header)
    cell.font = Font(bold=True)
    cell.fill = PatternFill(start_color='D3D3D3', end_color='D3D3D3', fill_type='solid')
    cell.alignment = Alignment(horizontal='center')

# Add data
for row_idx, (_, row) in enumerate(summary_df.iterrows(), 8):
    ws.cell(row=row_idx, column=1, value=row['Class'])
    ws.cell(row=row_idx, column=2, value=row['Range'])
    ws.cell(row=row_idx, column=3, value=row['Entry_Count'])
    ws.cell(row=row_idx, column=4, value=row['Entry_Percentage'])
    ws.cell(row=row_idx, column=5, value=row['Point_Count_Sum'])
    ws.cell(row=row_idx, column=6, value=row['Point_Count_Percentage'])

# Auto-adjust column widths
for col_num in range(1, 7):  # Columns A to F
    max_length = 0
    column_letter = get_column_letter(col_num)

    # Check all cells in this column
    for row_num in range(1, ws.max_row + 1):
        cell = ws.cell(row=row_num, column=col_num)
        try:
            if cell.value and len(str(cell.value)) > max_length:
                max_length = len(str(cell.value))
        except:
            pass

    # Set column width
    adjusted_width = min(max_length + 2, 25)
    ws.column_dimensions[column_letter].width = adjusted_width

# Add borders
thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

for row in ws.iter_rows(min_row=7, max_row=7 + len(summary_df), min_col=1, max_col=6):
    for cell in row:
        cell.border = thin_border
        # Center align numeric columns
        if cell.column in [1, 3, 4, 5, 6]:
            cell.alignment = Alignment(horizontal='center')

# Save the file
excel_filename = "point_count_analysis.xlsx"
wb.save(excel_filename)
print(f"\nExcel file saved as: {excel_filename}")


Excel file saved as: point_count_analysis.xlsx


In [26]:
import geopandas as gpd
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

# Load data
data = gpd.read_file(r"C:\Users\meloy\PycharmProjects\WebGisGeneratorV1\Data\Missouri\Missouri BEAD Grid Analysis Layer.sqlite")

# Define ranges
range_dict = {
    "0": {"range": (1, 5), "color": "#e4e4f3"},
    "1": {"range": (5, 10), "color": "#d1d1ea"},
    "2": {"range": (10, 20), "color": "#b3b3e0"},
    "3": {"range": (20, 30), "color": "#8080c5"},
    "4": {"range": (30, 50), "color": "#6d6dbd"},
    "5": {"range": (50, 75), "color": "#4949ac"},
    "6": {"range": (75, 100), "color": "#3737a4"},
    "7": {"range": (100, 50000), "color": "#121293"},
}

def classify_point_count(point_count):
    for class_key, class_info in range_dict.items():
        min_val, max_val = class_info["range"]
        if min_val <= point_count < max_val:
            return class_key
    return "unclassified"

# Classify
data['range_class'] = data['point_count'].apply(classify_point_count)
total_entries = len(data)
total_point_count = data['point_count'].sum()

# Prepare summary
summary_data = []
for class_key, class_info in range_dict.items():
    min_val, max_val = class_info["range"]
    class_data = data[data['range_class'] == class_key]
    count = len(class_data)
    percentage = (count / total_entries) * 100
    point_count_sum = class_data['point_count'].sum()
    point_count_percentage = (point_count_sum / total_point_count) * 100
    display_max = max_val - 1 if max_val != 50000 else "50000+"
    range_display = f"{min_val} - {display_max}"
    average = round(point_count_sum / count) if count > 0 else None

    summary_data.append({
        'Class': int(class_key),
        'Range': range_display,
        'Entry Count (1 Sq Km Grids)': count,
        'Entry Percentage (%)': f"{round(percentage, 1)}%",
        'Point Count (BSLs) Sum': point_count_sum,
        'Point Count Percentage (%)': f"{round(point_count_percentage, 1)}%",
        'Average Locations per Box': average
    })

# Add unclassified
unclassified_data = data[data['range_class'] == 'unclassified']
if len(unclassified_data) > 0:
    count = len(unclassified_data)
    percentage = (count / total_entries) * 100
    summary_data.append({
        'Class': 'Unclassified',
        'Range': 'Zero Locations',
        'Entry Count (1 Sq Km Grids)': count,
        'Entry Percentage (%)': f"{round(percentage, 1)}%",
        'Point Count (BSLs) Sum': None,
        'Point Count Percentage (%)': None,
        'Average Locations per Box': None
    })

summary_df = pd.DataFrame(summary_data)

# Write to Excel
wb = Workbook()
ws = wb.active
ws.title = "Point Count Analysis"

# Title
ws.merge_cells('A1:G1')
ws['A1'] = "Point Count Distribution Analysis"
ws['A1'].font = Font(bold=True, size=14)
ws['A1'].alignment = Alignment(horizontal='center')

# Header Info
ws['A3'] = "Missouri"
ws['A4'] = f"Total Entries (1 km sq Grids): {total_entries:,}"
ws['A4'].font = Font(bold=True)
ws['A5'] = f"Total Point Count (BSL): {total_point_count:,}"
ws['A5'].font = Font(bold=True)
ws['A6'] = f"Analysis Date: {pd.Timestamp.now().strftime('%Y-%m-%d')}"

# Headers
headers = list(summary_df.columns)
for col_idx, header in enumerate(headers, start=1):
    cell = ws.cell(row=8, column=col_idx, value=header)
    cell.font = Font(bold=True)
    cell.fill = PatternFill(start_color='D3D3D3', end_color='D3D3D3', fill_type='solid')
    cell.alignment = Alignment(horizontal='center')

# Data
for row_idx, row in enumerate(summary_df.itertuples(index=False), start=9):
    for col_idx, value in enumerate(row, start=1):
        cell = ws.cell(row=row_idx, column=col_idx, value=value)

        # Comma formatting
        if headers[col_idx - 1] in ['Entry Count (1 Sq Km Grids)', 'Point Count (BSLs) Sum', 'Average Locations per Box'] and isinstance(value, (int, float)):
            cell.number_format = '#,##0'

        # Center align all cells
        cell.alignment = Alignment(horizontal='center')

# Auto-fit columns
for col_idx in range(1, len(headers) + 1):
    max_length = 0
    column_letter = get_column_letter(col_idx)
    for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=col_idx, max_col=col_idx):
        for cell in row:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
    ws.column_dimensions[column_letter].width = min(max_length + 2, 30)

# Add borders
thin_border = Border(left=Side(style='thin'), right=Side(style='thin'),
                     top=Side(style='thin'), bottom=Side(style='thin'))
for row in ws.iter_rows(min_row=8, max_row=8 + len(summary_df), min_col=1, max_col=len(headers)):
    for cell in row:
        cell.border = thin_border

# Save
wb.save("point_count_analysis_formatted.xlsx")
print("Excel file saved as: point_count_analysis_formatted.xlsx")

Excel file saved as: point_count_analysis_formatted.xlsx


In [33]:
import geopandas as gpd
import pandas as pd
import os
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

# Define the folder containing all state subfolders
root_dir = r"C:\Users\meloy\SW2020 Dropbox\SW2020\Workspaces\Narek_Meloyan\Tarana Project\Data-Structured"

# Define range dictionary once
range_dict = {
    "0": {"range": (1, 5)},
    "1": {"range": (5, 10)},
    "2": {"range": (10, 20)},
    "3": {"range": (20, 30)},
    "4": {"range": (30, 50)},
    "5": {"range": (50, 75)},
    "6": {"range": (75, 100)},
    "7": {"range": (100, 50000)},
}

# Classification function
def classify_point_count(point_count):
    for class_key, class_info in range_dict.items():
        min_val, max_val = class_info["range"]
        if min_val <= point_count < max_val:
            return class_key
    return "unclassified"

# Process each state folder
for state_name in os.listdir(root_dir):
    state_path = os.path.join(root_dir, state_name)
    if not os.path.isdir(state_path):
        continue

    # Find the .sqlite file inside the folder
    sqlite_files = [f for f in os.listdir(state_path) if f.endswith(".sqlite") and f.startswith(f"{state_name} BEAD Grid Analysis Layer")]
    if not sqlite_files:
        print(f"[SKIPPED] No .sqlite file found in {state_name}")
        continue

    sqlite_path = os.path.join(state_path, sqlite_files[0])
    print(f"[PROCESSING] {state_name} — {sqlite_files[0]}")

    try:
        # Load data
        data = gpd.read_file(sqlite_path)
        # print(data.head())
        data['range_class'] = data['point_count'].apply(classify_point_count)

        total_entries = len(data)
        total_point_count = data['point_count'].sum()

        summary_data = []
        for class_key, class_info in range_dict.items():
            min_val, max_val = class_info["range"]
            class_data = data[data['range_class'] == class_key]
            count = len(class_data)
            percentage = (count / total_entries) * 100
            point_count_sum = class_data['point_count'].sum()
            point_count_percentage = (point_count_sum / total_point_count) * 100
            display_max = max_val - 1 if max_val != 50000 else "50000+"
            range_display = f"{min_val} - {display_max}"
            average = round(point_count_sum / count) if count > 0 else None

            summary_data.append({
                'Class': int(class_key),
                'Range': range_display,
                'Entry Count (1 Sq Km Grids)': count,
                'Entry Percentage (%)': f"{round(percentage, 1)}%",
                'Point Count (BSLs) Sum': point_count_sum,
                'Point Count Percentage (%)': f"{round(point_count_percentage, 1)}%",
                'Average Locations per Box': average
            })

        # Add unclassified
        unclassified_data = data[data['range_class'] == 'unclassified']
        if len(unclassified_data) > 0:
            count = len(unclassified_data)
            percentage = (count / total_entries) * 100
            summary_data.append({
                'Class': 'Unclassified',
                'Range': 'Zero Locations',
                'Entry Count (1 Sq Km Grids)': count,
                'Entry Percentage (%)': f"{round(percentage, 1)}%",
                'Point Count (BSLs) Sum': None,
                'Point Count Percentage (%)': None,
                'Average Locations per Box': None
            })

        summary_df = pd.DataFrame(summary_data)

        # Create Excel
        wb = Workbook()
        ws = wb.active
        ws.title = "Point Count Analysis"
        ws.merge_cells('A1:G1')
        ws['A1'] = "Point Count Distribution Analysis"
        ws['A1'].font = Font(bold=True, size=14)
        ws['A1'].alignment = Alignment(horizontal='center')

        ws['A3'] = state_name
        ws['A4'] = f"Total Entries (1 km sq Grids): {total_entries:,}"
        ws['A4'].font = Font(bold=True)
        ws['A5'] = f"Total Point Count (BSL): {total_point_count:,}"
        ws['A5'].font = Font(bold=True)
        ws['A6'] = f"Analysis Date: {pd.Timestamp.now().strftime('%Y-%m-%d')}"

        headers = list(summary_df.columns)
        for col_idx, header in enumerate(headers, start=1):
            cell = ws.cell(row=8, column=col_idx, value=header)
            cell.font = Font(bold=True)
            cell.fill = PatternFill(start_color='D3D3D3', end_color='D3D3D3', fill_type='solid')
            cell.alignment = Alignment(horizontal='center')

        for row_idx, row in enumerate(summary_df.itertuples(index=False), start=9):
            for col_idx, value in enumerate(row, start=1):
                cell = ws.cell(row=row_idx, column=col_idx, value=value)
                if headers[col_idx - 1] in ['Entry Count (1 Sq Km Grids)', 'Point Count (BSLs) Sum', 'Average Locations per Box'] and isinstance(value, (int, float)):
                    cell.number_format = '#,##0'
                cell.alignment = Alignment(horizontal='center')

        # Auto-fit columns
        for col_idx in range(1, len(headers) + 1):
            max_length = 0
            column_letter = get_column_letter(col_idx)
            for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=col_idx, max_col=col_idx):
                for cell in row:
                    if cell.value:
                        max_length = max(max_length, len(str(cell.value)))
            ws.column_dimensions[column_letter].width = min(max_length + 2, 30)

        thin_border = Border(left=Side(style='thin'), right=Side(style='thin'),
                             top=Side(style='thin'), bottom=Side(style='thin'))
        for row in ws.iter_rows(min_row=8, max_row=8 + len(summary_df), min_col=1, max_col=len(headers)):
            for cell in row:
                cell.border = thin_border

        # Save Excel in same folder
        output_path = os.path.join(state_path, "point_count_analysis.xlsx")
        wb.save(output_path)
        print(f"[SAVED] {state_name} → {output_path}")

    except Exception as e:
        print(f"[ERROR] Failed to process {state_name}: {e}")


[PROCESSING] Alabama — Alabama BEAD Grid Analysis Layer.sqlite
[SAVED] Alabama → C:\Users\meloy\SW2020 Dropbox\SW2020\Workspaces\Narek_Meloyan\Tarana Project\Data-Structured\Alabama\point_count_analysis.xlsx
[SKIPPED] No .sqlite file found in Alaska
[PROCESSING] Arizona — Arizona BEAD Grid Analysis Layer.sqlite
[SAVED] Arizona → C:\Users\meloy\SW2020 Dropbox\SW2020\Workspaces\Narek_Meloyan\Tarana Project\Data-Structured\Arizona\point_count_analysis.xlsx
[PROCESSING] Arkansas — Arkansas BEAD Grid Analysis Layer.sqlite
[SAVED] Arkansas → C:\Users\meloy\SW2020 Dropbox\SW2020\Workspaces\Narek_Meloyan\Tarana Project\Data-Structured\Arkansas\point_count_analysis.xlsx
[PROCESSING] California — California BEAD Grid Analysis Layer.sqlite
[SAVED] California → C:\Users\meloy\SW2020 Dropbox\SW2020\Workspaces\Narek_Meloyan\Tarana Project\Data-Structured\California\point_count_analysis.xlsx
[PROCESSING] Colorado — Colorado BEAD Grid Analysis Layer.sqlite
[SAVED] Colorado → C:\Users\meloy\SW2020 Drop